   ## Aaconda 更新
   Open Anaconda Prompt, **run as admin**
   > conda update --all
   
   ## NLTK 安装

In [1]:
import nltk
#nltk.set_proxy('http://127.0.0.1:1080', ('USERNAME', 'PASSWORD'))
nltk.set_proxy('http://proxy:8080', ('USERNAME', 'PASSWORD'))
#nltk.download()

In [2]:
# 或者，直接指定下载哪个
# nltk.download('popular')

会弹出来一个窗口：
- Collections
- Corpora 语料
- Models 模型
- All Packages

## NLTK 自带语料库

In [3]:
# 如果之前没有下载过 brown 语料库，可以临时下载
# nltk.download('brown')

# brown大学的语料库，包含很多分类
from nltk.corpus import brown

# 如果遇到 "No Disc" error
# Navigate to: HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Windows\ 
# Change the value of the ErrorMode key to 2.

# 看看包含多少分类
print(brown.categories())

# 看看多少句子，多少单词
print(len(brown.sents()))
print(len(brown.words()))

['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies', 'humor', 'learned', 'lore', 'mystery', 'news', 'religion', 'reviews', 'romance', 'science_fiction']
57340
1161192


## 文本处理流程
- Preprocess 预处理
  - 包含 Tokenize 分词，等等其它步骤
- 生成 Features 特征 (as X)
- 机器学习

## Tokenize
把长句拆成有意义的小部件

In [4]:
import nltk
sentence = "hello, world"
tokens = nltk.word_tokenize(sentence)
tokens

['hello', ',', 'world']

## 中英文NLP区别
例句：今天/天气/不错
- 启发式 Heuristic，寻找最长的拟合词
- 机器学习/统计方法：HMM（隐马尔科夫链），CRF
  - 斯坦福的 CoreNLP 可以支持中文
  
## 中文分词
- 安装jieba:

Open Anaconda Prompt
> pip install --proxy 127.0.0.1:1080 jieba

or
> pip install jieba

也可以研究下CoreNLP，也有中文分词功能

In [5]:
import jieba
seg_list = jieba.cut("我来到北京清华大学", cut_all=True) # 全模式
"/ ".join(seg_list)

Building prefix dict from the default dictionary ...
Loading model from cache D:\Temp\jieba.cache
Loading model cost 0.830 seconds.
Prefix dict has been built succesfully.


'我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学'

In [6]:
seg_list = jieba.cut("我来到北京清华大学", cut_all=False) # 精确模式
"/ ".join(seg_list)

'我/ 来到/ 北京/ 清华大学'

In [7]:
seg_list = jieba.cut("他来到了网易杭研大厦") # 默认精确模式
"/ ".join(seg_list)
# 注意：这里“杭研”并没有在词典中，但也被Viterbi算法识别出来了

'他/ 来到/ 了/ 网易/ 杭研/ 大厦'

In [8]:
seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造") # 搜索引擎模式
"/ ".join(seg_list)
# 该模式所有可能的都排列出来了，更适合于搜索引擎的使用

'小明/ 硕士/ 毕业/ 于/ 中国/ 科学/ 学院/ 科学院/ 中国科学院/ 计算/ 计算所/ ，/ 后/ 在/ 日本/ 京都/ 大学/ 日本京都大学/ 深造'

## 有时候分词没有那么简单
例如：
- RT是转发
- @
- :) 笑脸
- 等等

In [9]:
from nltk.tokenize import word_tokenize
tweet = 'RT @angelababy: love you baby! :D http://ah.love #168cm'
print(word_tokenize(tweet))

['RT', '@', 'angelababy', ':', 'love', 'you', 'baby', '!', ':', 'D', 'http', ':', '//ah.love', '#', '168cm']


## 社交语言的 tokenize

对于下面的`emoticons_str`，例子：
- `:-)`
- `:)`

正则对照表: http://www.regexlab.com/zh/regref.htm

In [10]:
import re

emoticons_str = r"""
    (?:
        [:=;] # 眼睛
        [oO\-]? # 鼻子
        [D\)\]\(\]/\\OpP] # 嘴
    )"""

regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # at 某人
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # 话题标签
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # 数字
    r"(?:[a-z][a-z'\-_]+[a-z])", # 含有 - 和 ‘ 的单词，例如 don't
    r'(?:[\w_]+)', # 其他
    r'(?:\S)' # 其他
]

In [11]:
tokens_re = re.compile(r'(' + '|'.join(regex_str) + ')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase: # 对于表情，需要让“是否小写”这个参数不生效
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

tweet = 'RT @angelababy: love you baby! :D http://ah.love #168cm'
print(preprocess(tweet))

['RT', '@angelababy', ':', 'love', 'you', 'baby', '!', ':D', 'http://ah.love', '#168cm']


这些特殊符号是否保留，需要根据你的任务来判断

## 纷繁复杂的词形（英文）

对于英文，从语素构成单词的方法主要有两大类(可能部分交叉)：inflection(屈折)和 derivation(派生)
- Infection (屈折)
  - 不影响词性 walk => walking => walked

- Derivation
  - 影响词性 nation => national => nationalize

简单一点，对于英文，直接依赖 `word_tokenize`，寄希望语料库足够大。但是为了效果更好，需要更好的预处理。

## 词形归一化（英文）

- Stemming 词⼲提取，⼀般来说，就是把不影响词性的inflection的⼩尾巴砍掉，例如：
  - walking => walk
  - walked => walk

- Lemmatization 词形归⼀：把各种类型的词的变形，都归为⼀个形式。通过 wordnet 的网络，一个语料库，实现
  - went => go
  - are => be
  
## NLTK实现Stemming
NLTK提供了不同的词干提取的类，各自有自己的规则，偷懒的可以用 SnowBallStemmer

In [12]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
print(porter_stemmer.stem('maximum'))
print(porter_stemmer.stem('presumably'))
print(porter_stemmer.stem('multiply'))
print(porter_stemmer.stem('provision'))

maximum
presum
multipli
provis


In [13]:
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
print(lancaster_stemmer.stem('maximum'))
print(lancaster_stemmer.stem('presumably'))
print(lancaster_stemmer.stem('multiply'))
print(lancaster_stemmer.stem('provision'))

maxim
presum
multiply
provid


In [14]:
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer("english")
print(snowball_stemmer.stem('maximum'))
print(snowball_stemmer.stem('presumably'))
print(snowball_stemmer.stem('multiply'))
print(snowball_stemmer.stem('provision'))

maximum
presum
multipli
provis


## NLTK实现Lemma（词形归一）

In [15]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
print(wordnet_lemmatizer.lemmatize('dogs'))
print(wordnet_lemmatizer.lemmatize('churches'))
print(wordnet_lemmatizer.lemmatize('aardwolves'))
print(wordnet_lemmatizer.lemmatize('abaci'))
print(wordnet_lemmatizer.lemmatize('hardrock'))

dog
church
aardwolf
abacus
hardrock


## Lemma的⼩问题

例如，went是go的过去式，但也同时是个英文名。
这里需要我们来告诉算法词性。

**POS**: part of speech，即：在我的这句话中，词性是什么

In [16]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
print(wordnet_lemmatizer.lemmatize('went')) #默认参数是 'n'
print(wordnet_lemmatizer.lemmatize('went', pos='n'))
print(wordnet_lemmatizer.lemmatize('went', pos='v'))


went
went
go


In [17]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
print(wordnet_lemmatizer.lemmatize('are')) # are 作为名词，没有此单词，返回自己
print(wordnet_lemmatizer.lemmatize('is')) # is 作为名词，没有此单词，返回自己

print(wordnet_lemmatizer.lemmatize('are', pos='v'))
print(wordnet_lemmatizer.lemmatize('is', pos='v'))

are
is
be
be


## NLTK标注 POS Tag
类似的，中文库中也有，例如jieba

In [18]:
import nltk
text = nltk.word_tokenize('what does the fox say')
print(text)
print(nltk.pos_tag(text))

['what', 'does', 'the', 'fox', 'say']
[('what', 'WDT'), ('does', 'VBZ'), ('the', 'DT'), ('fox', 'NNS'), ('say', 'VBP')]


上面词性的输出可以映射到简单的词性，例如 'NNS' => 'n'，提供给lemmatize函数进行归一化

## Stopwords

- 中文，例如 的,得,地
- 英文，例如 the

容易导致歧义

英文停止词列表：http://www.ranks.nl/stopwords
中文需要搜索

In [19]:
# 需要 先下载
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to D:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [21]:
from nltk.corpus import stopwords
word_list = nltk.word_tokenize("He is a good guy")
filtered_words =  [word for word in word_list if word not in stopwords.words('english')]
print(filtered_words)

['He', 'good', 'guy']


## ⼀条typical的⽂本预处理流⽔线

- Raw_Text
- Tokenize
  - POS Tag
- Lemma/Stemming
- Stopwords
- Word_List

对于场景不同，上述步骤可能有所不同。
- 例如，判断一个人的写作能力怎么样，或者判断文章的相似度，不能去除停止词，因为破坏了句子结构
- 但是，加入用于 word2vec，用于判断词义，大家都去除停止词比较方便

## NLTK在NLP上的经典应用

- 情感分析 
- ⽂本相似度 
- ⽂本分类

## 情感分析

哪些是夸你，哪些是黑你

最简单的是 sentiment dictionary，类似于关键词的打分机制
- 简单，不需要机器学习
- 应用相对广泛
- ⽐如：AFINN-111 http://www2.imm.dtu.dk/pubdb/views/publication_details.php?id=6010

In [22]:
sentiment_dictionary = {}
for line in open('data/AFINN-111.txt'):
    word, score = line.split('\t')
    sentiment_dictionary[word] = int(score)

words = nltk.word_tokenize("This is a good book")
words =  [word for word in words if word not in stopwords.words('english')]
print(words)

# 把这个打分表记录在⼀个Dict上以后
# 跑一遍整个句⼦，把对应的值相加
total_score = sum(sentiment_dictionary.get(word, 0) for word in words)
# 有值就是Dict中的值，没有就是0
# 于是你就得到了⼀个 sentiment score
total_score

['This', 'good', 'book']


3

该方法的问题：
- 新词怎么办
- 特殊词怎么办
- 更深层次的玩意儿怎么办

## 配上ML的情感分析

下面的例子可以这样理解，
- 有一组单词：
`['this', 'is', 'a', 'good', 'awesome', 'bad', 'terrible', 'book']`。
- 预处理后输入的其实把一句话中用一个向量表示。例如：
`['this': True, 'is': True, 'a': True, 'good': True, 'awesome': False, 'bad': False, 'terrible': False, 'book': True]`

In [23]:
from nltk.classify import NaiveBayesClassifier

# 随⼿手造点训练集
s1 = 'this is a good book'
s2 = 'this is a awesome book'
s3 = 'this is a bad book'
s4 = 'this is a terrible book'

def preprocess(s):
    # Func: 句⼦处理
    # 这⾥简单的⽤了split(), 把句子中每个单词分开
    # 显然 还有更多的processing method可以用
    return {word: True for word in s.lower().split()}
    # return⻓这样:
    # {'this': True, 'is':True, 'a':True, 'good':True, 'book':True}
    # 其中, 前⼀个叫fname, 对应每个出现的⽂本单词;
    # 后⼀个叫fval, 指的是每个⽂本单词对应的值。
    # 这⾥我们⽤最简单的True,来表示,这个词『出现在当前的句子中』的意义。
    # 当然啦, 我们以后可以升级这个⽅程, 让它带有更加⽜逼的fval, ⽐如 word2vec

# 把训练集给做成标准形式
training_data = [[preprocess(s1), 'pos'],
                 [preprocess(s2), 'pos'],
                 [preprocess(s3), 'neg'],
                 [preprocess(s4), 'neg']]
# 喂给model吃
model = NaiveBayesClassifier.train(training_data)
# 打出结果
print(model.classify(preprocess('this is a good book')))

pos
